In [2]:
import torch
from torch import nn
from torch.optim import Adam, SGD, RMSprop
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset


In [3]:
from torchvision.datasets.fakedata import transforms
train= datasets.MNIST(root='data', download=True, train=True, transform=ToTensor())
test= datasets.MNIST(root='data',download=True, train=False, transform=ToTensor())

dataset_train = DataLoader(train, batch_size=32, shuffle=True)
dataset_test = DataLoader(test, batch_size=32, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [47]:
##Darknet-19 model

class BlockConv1(nn.Module):
  def __init__(self, input_channels=56, num_kernel=28, size_stride=1, *kwgars):
    super().__init__()
    self.conv1 = nn.Conv2d(input_channels,input_channels, kernel_size=num_kernel*2, stride=size_stride*3)
    self.conv2 = nn.Conv2d(input_channels,input_channels, kernel_size=num_kernel, stride=(size_stride))
    self.conv3 = nn.Conv2d(input_channels,input_channels, kernel_size=num_kernel*2, stride=size_stride*3)
    self.max_pool=nn.MaxPool2d(kernel_size=2, stride=2)

  def forward(self,x):
    X = self.conv1(x)
    X= self.conv2(X)
    X = self.conv3(X)
    X = self.conv3(X)
    return X

class DarkNetClassifier(nn.Module):
  def __init__(self, Block1Conv1, input_channels, output_channels=224, num__kernels=32,  num_stride=1, num_classes=10):
    super(DarkNetClassifier, self).__init__()

    self.conv1 = nn.Conv2d(input_channels,output_channels, kernel_size = num__kernels, stride = num_stride*3, bias=False)
    self.max_pool1=nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(output_channels,output_channels, kernel_size = (2*num__kernels), stride = num_stride*3, bias=False)
    self.max_pool2=nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.block1 = BlockConv1(input_channels=56,num_kernel=128, size_stride=num_stride)
    self.block2 = BlockConv1(input_channels=28,num_kernel=128, size_stride=num_stride)

    self.conv3 = nn.Conv2d(14,14, kernel_size = 512, stride = num_stride*3, bias=False)
    self.conv4 = nn.Conv2d(14,14, kernel_size = 256, stride = (num_stride), bias=False)
    self.conv5 = nn.Conv2d(14,14, kernel_size = 512, stride = num_stride*3, bias=False)
    self.conv6 = nn.Conv2d(14,14, kernel_size = 256, stride = (num_stride), bias=False)
    self.conv7 = nn.Conv2d(14,14, kernel_size = 512, stride = num_stride*3, bias=False)
    self.max_pool3=nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv8 = nn.Conv2d(7,7, kernel_size = 1024, stride = num_stride*3, bias=False)
    self.conv9 = nn.Conv2d(7,7, kernel_size = 512, stride = num_stride, bias=False)
    self.conv10 = nn.Conv2d(7,7, kernel_size = 1024, stride = num_stride*3, bias=False)
    self.conv11 = nn.Conv2d(7,7, kernel_size = 512, stride = (num_stride), bias=False)
    self.conv12 = nn.Conv2d(7,7, kernel_size = 1024, stride = num_stride*3, bias=False)

    self.conv_last_layer=nn.Conv2d(7,7,kernel_size=num_classes, stride = num_stride, bias=False)
    self.avg= nn.AvgPool2d(num_classes)
    self.soft=nn.Softmax(dim=1)

  def forward(self,x):
    model= self.soft(self.avg(self.conv_last_layer(self.conv12(self.conv11(self.conv10(self.conv9(self.conv8(self.max_pool3(self.conv7(self.conv6(self.conv5(self.conv4(self.conv3(self.block2(self.block1(self.max_pool2(self.conv2(self.max_pool1(self.conv1(x))))))))))))))))))))
    return x

def test():
  model=DarkNetClassifier(BlockConv1, input_channels=224, output_channels=224, num__kernels=32,  num_stride=1, num_classes=10)
  X=torch.rand(2,3,224,224)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
  Y = model.to(device)
  print(Y)

In [48]:
test()

DarkNetClassifier(
  (conv1): Conv2d(224, 224, kernel_size=(32, 32), stride=(3, 3), bias=False)
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(224, 224, kernel_size=(64, 64), stride=(3, 3), bias=False)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block1): BlockConv1(
    (conv1): Conv2d(56, 56, kernel_size=(256, 256), stride=(3, 3))
    (conv2): Conv2d(56, 56, kernel_size=(128, 128), stride=(1, 1))
    (conv3): Conv2d(56, 56, kernel_size=(256, 256), stride=(3, 3))
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): BlockConv1(
    (conv1): Conv2d(28, 28, kernel_size=(256, 256), stride=(3, 3))
    (conv2): Conv2d(28, 28, kernel_size=(128, 128), stride=(1, 1))
    (conv3): Conv2d(28, 28, kernel_size=(256, 256), stride=(3, 3))
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv